# __Netwerkanalyse Nederlands spoorwegennetwerk (NS; 2024)__

<tekst>dddd

In [26]:
import networkx as nx
import polars as pl
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import statistics
from geopandas import GeoDataFrame
from matplotlib.patches import Patch

from coordinaat import coordinaatstring_naar_exact
from graaf import maak_stationsgraaf, voeg_verbindingen_toe, aantal_verbindingen_nodig_voor_complete_graaf
from station import geef_hoofdstations, geef_inwoners_en_stations_provincies

### 1. Het inlezen van de dataverzameling

In [2]:
stations: pl.DataFrame = pl.read_csv("data/dataStationsStedenNederland.csv", encoding="latin-1")
stations.head()

### 2. Het verrijken van de stationsdata

In [3]:
coordinaten_stations: dict[str, tuple[float, float]] = {
    station: coordinaatstring_naar_exact(coordinaat) for station, coordinaat in zip(stations["station"], stations["coordinaten"])
}

In [4]:
S: nx.DiGraph = maak_stationsgraaf(coordinaten_stations)
S = voeg_verbindingen_toe(S, stations)

In [5]:
HOOFDSTATIONS: set[str] = geef_hoofdstations(S)

### 3. Visualisatie Nederlandse spoorwegen (NS)

Technische begrippen:
- _knoop_: een entiteit in een graaf;
- _graad_: het aantal verbindingen/buren van een knoop;
- _bereik_: hoeveel andere knopen een knoop kan bereiken via haar verbindingen en buren;
- _clustervormingscoëfficiënt_: de kans de de buren van een knoop ook onderling verbindingen hebben;
- _dichtheid_: de ratio tussen het aantal bestaande en mogelijke verbindingen;
- _subgraaf_: een kleinere graaf in een grotere graaf;
- _egograaf_: een subgraaf van een specifieke knoop die louter de knoop zelf en haar buren bevat;
- _kliek_: een subgraaf waarin iedere knoop een verbinding heeft met iedere andere knoop — behalve met zichzelf;
- _complete graaf_: een graaf waarin iedere knoop in verbinding heeft met iedere andere — behalve met zichzelf: compleet = $\frac{N \cdot (N - 1)}{2}$
;
- _betweenneesscentrality_: de tussenliggenheid van een knoop op kortste paden tussen andere knopen;
- _grootste-effectknoop_: de knoop die de meeste chaos zou veroorzaken, als die zou komen te ontvallen.

In [6]:
os.environ["SHAPE_RESTORE_SHX"] = "YES"

kaart_nederland: GeoDataFrame = gpd.read_file("data/NLD_adm0.shp")

figuur, as_ = plt.subplots(figsize=(30, 30))
kaart_nederland.plot(ax=as_, color="lightgrey")
stationspositities: dict[str, tuple[float, float]] = nx.get_node_attributes(S, "pos")
stationspositities = {station: (coordinaten[0], coordinaten[1]) for station, coordinaten in stationspositities.items()}

stationskleuren: list[str] = ["darkblue" if (station in HOOFDSTATIONS) else "red" for station in S]
stationsgrootten: list[int] = [90 if (station in HOOFDSTATIONS) else 50 for station in S]

legendalabel_hoofdstations: Patch = Patch(color="darkblue", label="Hoofdstation")
legendalabel_tussenstation: Patch = Patch(color="red", label="Tussenstation")

nx.draw_networkx_nodes(S, stationspositities, node_size=stationsgrootten, node_color=stationskleuren, ax=as_)
nx.draw_networkx_edges(S, stationspositities, ax=as_)
nx.draw_networkx_labels(S, stationspositities, font_size=10, font_color="black", ax=as_)

plt.title("Nederlandse spoorwegen (NS) — stations en verbindingen\n")
plt.xlabel("Lengtegraad")
plt.ylabel("Breedtegraad")
plt.show()

### 4. Beschrijvende waarden Nederlands spoorwegennetwerk


In [7]:
BUITENLANDSE_STATIONS: set[str] = {
    "Herzogenrath", "Visé", "Antwerpen-Centraal", "Essen", "Emmerich-Elten", "Gronau",
    "Bad Bentheim", "Weener", "Breyell", "Noorderkempen", "Mönchengladbach"
}

aantal_inwoners_per_provincie: dict[str, int] = {
    "Drenthe": 504_116, "Flevoland": 450_920, "Friesland": 661_956, "Gelderland": 2_149_057,
    "Groningen": 601_510, "Limburg": 1_133_174, "Noord-Brabant": 2_644_872, "Noord-Holland": 2_980_494,
    "Overijssel": 1_189_079, "Utrecht": 1_400_187, "Zeeland": 391_657, "Zuid-Holland":	3_840_662	
}

aantal_inwoners_en_stations_per_provincie: dict[str, list[int | set[str]]] = geef_inwoners_en_stations_provincies(stations, aantal_inwoners_per_provincie)

In [23]:
bereiken_stations: dict[str, int] = {station1: sum(1 for station2 in S if nx.has_path(S, station1, station2)) for station1 in S}

minimumbereik: int = min(bereiken_stations.values())
maximumbereik: int = max(bereiken_stations.values())
gemiddelde_bereik_station: float = statistics.mean(list(bereiken_stations.values()))

uitgraden_stations: list[int] = [uitgraad for station, uitgraad in S.out_degree()]
ingraden_stations: list[int] = [ingraad for station, ingraad in S.in_degree()]
gemiddelde_uitgraad_stations: float = statistics.mean(uitgraden_stations)
gemiddelde_ingraad_stations: float = statistics.mean(ingraden_stations)

TUSSENSTATIONS: set[str] = {station for station in S if not (station in HOOFDSTATIONS)}
nederlandsche_stations: set[str] = {station for station in S if not (station in BUITENLANDSE_STATIONS)}

aantallen_stations_per_provincie: list = [len(stationsgegevens[1]) for stationsgegevens in aantal_inwoners_en_stations_per_provincie.values()]
aantallen_hoofdstations_per_provincie: list = [
    len({station for station in stationsgegevens[1] if station in HOOFDSTATIONS})
    for stationsgegevens in aantal_inwoners_en_stations_per_provincie.values()
]
aantallen_tussenstations_per_provincie: list = [
    len({station for station in stationsgegevens[1] if station in TUSSENSTATIONS})
    for stationsgegevens in aantal_inwoners_en_stations_per_provincie.values()
]

aantal_symmetrische_verbingen: int = sum(1 for station1, station2 in S.edges() if S.has_edge(station2, station1))
stations_met_buitenlandse_verbindingen: set[str] = set()

for station in S:
    for buitenlands_station in BUITENLANDSE_STATIONS:
        if S.has_edge(station, buitenlands_station):
            stations_met_buitenlandse_verbindingen.add(station)

print(f"GRAAFGEGEVENS\n{'-' * 78}")
print(f"- aantal stations: {len(S)};")
print(f"- aantal Nederlandse stations: {len(nederlandsche_stations)};")
print(f"- aantal buitenlandse (Duitse en Belgische) stations: {len(BUITENLANDSE_STATIONS)};")
print(f"- aantal hoofdstations: {len(HOOFDSTATIONS)};")
print(f"- aantal tussenstations: {len(TUSSENSTATIONS)};")
print(f"- gemiddeld aantal stations per provincie: {statistics.mean(aantallen_stations_per_provincie)};")
print(f"- gemiddeld aantal hoofdstations per provincie: {round(statistics.mean(aantallen_hoofdstations_per_provincie), 2)};")
print(f"- gemiddeld aantal tussenstations per provincie: {round(statistics.mean(aantallen_tussenstations_per_provincie), 2)};\n")

print(f"- aantal verbindingen tussen stations: {len(S.edges())};")
print(f"- aantal symmetrische verbindingen tussen stations: {aantal_symmetrische_verbingen};")
print(f"- aantal stations met (directe) buitenlandse verbindingen: {len(stations_met_buitenlandse_verbindingen)};")
print(f"- aantal verbindingen vereist om graaf compleet te maken: {aantal_verbindingen_nodig_voor_complete_graaf(S)};\n")

print(f"- gemiddelde uitgraad stations: {round(gemiddelde_uitgraad_stations, 3)};")
print(f"- gemiddelde ingraad stations: {round(gemiddelde_ingraad_stations, 3)};")
print(f"- minimumingraad stations: {min(ingraden_stations)};")
print(f"- maximumingraad stations: {max(ingraden_stations)};")
print(f"- minimumuitgraad stations: {min(uitgraden_stations)};")
print(f"- minimumuitgraad stations: {max(uitgraden_stations)};\n")

print(f"- minimumbereik stations: {minimumbereik};")
print(f"- maximumbereik stations: {maximumbereik};")
print(f"- gemiddeld bereik stations: {round(gemiddelde_bereik_station, 3)};")
print(f"- ieder Nederlands station kan ieder ander Nederlands station bereiken: {list(bereiken_stations.values()).count(len(nederlandsche_stations)) == len(S)};\n")

print(f"- clustervormingscoëfficiënt: {round(nx.average_clustering(S), 3)};")
print(f"- dichtheid: {round(nx.density(S), 3)};")
print(f"- diameter: {nx.diameter(S) if nx.is_strongly_connected(S) else "niet sterk verbonden"};")
print(f"- radius: {nx.radius(S) if nx.is_strongly_connected(S) else "niet sterk verbonden"}.")